# Import

In [ ]:
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# Parameter

In [ ]:
axisFontSize  = 14
titleFontSize = 20
Trace_Path = 'C:/Users/user/Desktop/라즈베리파이4분석(3-2)/cpu_trace'

# parameter
samplingFrequency = int(20e9)      # unit: Hz
windowLength      = int(1e4)      # unit: point

# Spectrogram & BandPass

In [ ]:
# Loop through all combinations
for x_index in tqdm(range(10), desc="x_index Progress"):
    for y_index in tqdm(range(10), desc="y_index Progress", leave=False):
        for tracenum in range(200):
            print("({}, {}, {})".format(x_index, y_index, tracenum))
            # Load data
            try:
                trigger = np.load(f'{Trace_Path}/cpu_trigger_{x_index}_{y_index}_{tracenum}.npy')
                trace = np.load(f'{Trace_Path}/cpu_trace_{x_index}_{y_index}_{tracenum}.npy')
            except FileNotFoundError:
                print(f"File not found: cpu_trigger_{x_index}_{y_index}_{tracenum}.npy or cpu_trace_{x_index}_{y_index}_{tracenum}.npy")
                continue

            # Spectrogram
            f, t, Sxx = signal.spectrogram(trace, fs=samplingFrequency, window=signal.get_window(window='hamming', Nx=windowLength))
            duration = trace.shape[0] / samplingFrequency

            # Plot
            plt.figure(figsize=(10, 13))
            plt.subplot(6, 1, 1)
            plt.title('Original Signal', fontsize=titleFontSize)
            plt.plot(np.linspace(0, duration, trace.shape[0]), trace, linewidth=0.5)
            plt.plot(np.linspace(0, duration, trigger.shape[0]), trigger / 3000, color='red')
            plt.xlim(0, duration)
            plt.ylabel('Voltage (V)', fontsize=axisFontSize)

            plt.subplot(6, 1, 2)
            plt.title('Spectrogram', fontsize=titleFontSize)
            plt.pcolormesh(t, f, Sxx)
            plt.xlim(0, duration)
            plt.ylabel('Frequency [Hz]', fontsize=axisFontSize)

            plt.subplot(6, 1, 3)
            plt.title('Spectrogram near clock freq', fontsize=titleFontSize)
            ind1 = len(f[f<= 950e6])
            ind2 = len(f[f<=1050e6])
            plt.pcolormesh(t, f[ind1:ind2], Sxx[ind1:ind2, :])
            plt.xlim(0, duration)
            plt.ylabel('Voltage (V)', fontsize=axisFontSize)

            plt.subplot(6, 1, 4)
            plt.title('Spectrogram band', fontsize=titleFontSize)
            ind1 = len(f[f<=870e6])
            ind2 = len(f[f<=930e6])
            plt.pcolormesh(t, f[ind1:ind2], Sxx[ind1:ind2, :])
            plt.xlim(0, duration)
            plt.ylabel('Voltage (V)', fontsize=axisFontSize)
            
            plt.subplot(6, 1, 5)
            plt.title('Spectrogram low freq1', fontsize=titleFontSize)
            ind = len(f[f<=100e6])
            plt.pcolormesh(t, f[:ind], Sxx[:ind, :])
            plt.xlim(0, duration)
            plt.ylabel('Voltage (V)', fontsize=axisFontSize)
                     
            plt.subplot(6, 1, 6)
            plt.title('Spectrogram low freq2', fontsize=titleFontSize)
            ind = len(f[f<=50e6])
            plt.pcolormesh(t, f[:ind], Sxx[:ind, :])
            plt.xlim(0, duration)
            plt.ylabel('Voltage (V)', fontsize=axisFontSize)
            plt.xlabel('Time [sec]', fontsize=axisFontSize)

            plt.tight_layout()
            # Show or save the plot
            plt.show()
